In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F
from uility import get_batch

In [3]:
import pandas as pd
import random
trainx=pd.read_csv('../data/x_1730_1024_3.csv')
trainy=pd.read_csv('../data/y_1730_1024_3.csv')
#trainx=pd.read_csv('../data/x_600_2_JN.csv')
#trainy=pd.read_csv('../data/y_600_2_JN.csv')
#x_test=pd.read_csv('../data/x_1772_1024_3.csv')
#y_test=pd.read_csv('../data/y_1772_1024_3.csv')
index =list(trainx.index)
random.Random(0).shuffle(index)
num=int(len(index)/5*3)
x_train,y_train=trainx.iloc[index[:num],:].values,trainy.iloc[index[:num],:].values
x_test,y_test=trainx.iloc[index[num:],:].values,trainy.iloc[index[num:],:].values
data_X=np.vstack([x_train,x_test])

In [4]:
N=1024
H1=400
H2=200
H3=100
drop_rate=0.1
out_dim=10
class Encoder(nn.Module):
    def __init__(self):
        super( Encoder,self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(N, H1),
            nn.ReLU(),
            nn.BatchNorm1d(H1),
            nn.Dropout(drop_rate),
            nn.Linear(H1,H2),
            nn.ReLU(),
            nn.BatchNorm1d(H2),
            nn.Dropout(drop_rate),
            nn.Linear(H2, H3),
            nn.ReLU(),
            nn.BatchNorm1d(H3),
            nn.Dropout(drop_rate)
        )
    def forward(self,x):
        encoded = self.encoder(x)
        return encoded
    
class Decoder(nn.Module):
    def __init__(self):
        super( Decoder,self).__init__()
        self.decoder = nn.Sequential(
            nn.Linear(H3,H2),
            nn.ReLU(),
            
            nn.Linear(H2,H1),
            nn.ReLU(),
            nn.Linear(H1,N),
        )
    def forward(self,x):
        decoded = self.decoder(x)
        return decoded
class Soft_class(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc= nn.Sequential(
            nn.Linear(H3,out_dim),)
    
    def forward(self,x):   
        x=F.log_softmax(self.fc(x),dim=1)
        return x
encoder=Encoder()
decoder=Decoder()
soft_class=Soft_class()
optimizer_en = torch.optim.Adam(encoder.parameters(), lr=1e-3,weight_decay=0.01)
optimizer_de = torch.optim.Adam(decoder.parameters(), lr=1e-3,weight_decay=0.01)
optimizer_sf = torch.optim.Adam(soft_class.parameters(), lr=1e-3,weight_decay=0.01)
loss_func = nn.MSELoss()

In [5]:
## 自编码

batch_size=128
EPOCH=10
for epoch in range(EPOCH):
    train_step=int(len(x_train)/batch_size)
    generate_data=get_batch(x_train,x_train,batch_size)
    for step in range(train_step):
        bx_,by_ =next(generate_data)
        bx=Variable(torch.Tensor(bx_))
        by=Variable(torch.Tensor(by_))     
        encoded =encoder(bx)        
        decoded=decoder(encoded)
        loss = loss_func(decoded, by)
        optimizer_de.zero_grad()
        optimizer_en.zero_grad()
        loss.backward()
        optimizer_de.step()
        optimizer_en.step()
        if step % 100 == 0:
            print('Epoch: ', epoch,'| train loss: %.4f' % loss.data.numpy())

Epoch:  0 | train loss: 0.0898
Epoch:  1 | train loss: 0.0788
Epoch:  2 | train loss: 0.0788
Epoch:  3 | train loss: 0.0788
Epoch:  4 | train loss: 0.0788
Epoch:  5 | train loss: 0.0788
Epoch:  6 | train loss: 0.0788
Epoch:  7 | train loss: 0.0788
Epoch:  8 | train loss: 0.0788
Epoch:  9 | train loss: 0.0788


In [9]:
#分类
batch_size=128
EPOCH=50
for epoch in range(EPOCH):
    train_step=int(len(x_train)/batch_size)
    generate_data=get_batch(x_train,y_train,batch_size)
    for step in range(train_step):
        bx_,by_ =next(generate_data)
        bx=Variable(torch.Tensor(bx_))
        by=Variable(torch.Tensor(by_.argmax(axis=1))).long()
        
        encoded =encoder(bx)        
        soft_=soft_class(encoded)
        loss = F.nll_loss(soft_,by)
        
        optimizer_sf.zero_grad()
        optimizer_en.zero_grad()
        loss.backward()
        optimizer_sf.step()
        optimizer_en.step()
        if step % 100 == 0:
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.numpy())

Epoch:  0 | train loss: 0.0751
Epoch:  1 | train loss: 0.0622
Epoch:  2 | train loss: 0.0807
Epoch:  3 | train loss: 0.0464
Epoch:  4 | train loss: 0.0619
Epoch:  5 | train loss: 0.0633
Epoch:  6 | train loss: 0.0537
Epoch:  7 | train loss: 0.0831
Epoch:  8 | train loss: 0.0931
Epoch:  9 | train loss: 0.0509
Epoch:  10 | train loss: 0.0448
Epoch:  11 | train loss: 0.0475
Epoch:  12 | train loss: 0.0474
Epoch:  13 | train loss: 0.0837
Epoch:  14 | train loss: 0.1495
Epoch:  15 | train loss: 0.0475
Epoch:  16 | train loss: 0.0507
Epoch:  17 | train loss: 0.0429
Epoch:  18 | train loss: 0.0881
Epoch:  19 | train loss: 0.0570
Epoch:  20 | train loss: 0.0627
Epoch:  21 | train loss: 0.0492
Epoch:  22 | train loss: 0.1045
Epoch:  23 | train loss: 0.0539
Epoch:  24 | train loss: 0.0458
Epoch:  25 | train loss: 0.0756
Epoch:  26 | train loss: 0.1052
Epoch:  27 | train loss: 0.0706
Epoch:  28 | train loss: 0.0727
Epoch:  29 | train loss: 0.0750
Epoch:  30 | train loss: 0.0426
Epoch:  31 | train

In [10]:
def  eval(x_test,y_test):
    x_test_=Variable(torch.Tensor(x_test))
    y_test_=Variable(torch.Tensor(y_test.argmax(axis=1))).long()
    encoded =encoder(x_test_)
    soft_=soft_class(encoded)
    acc=(soft_.argmax(axis=1)==y_test_).sum().numpy()/len(y_test)
    cost= F.nll_loss(soft_,y_test_).detach().numpy()
    return cost,acc
eval(x_test,y_test)

(array(0.1834402, dtype=float32), 0.9534632034632035)

a=autoencoder.state_dict()
model_dict.update(pretrained_dict)
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
cnn.load_state_dict(model_dict)